# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import numpy as np
from keras.datasets import cifar10
import tensorflow as tf
from keras.utils import np_utils

#Load the dataset:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [2]:
#shapes
print('shape of X_train',X_train.shape)
print('shape of X_test',X_test.shape)
print('shape of y_train',y_train.shape)
print('shape of y_test',y_test.shape)
n_examples = 50000

shape of X_train (50000, 32, 32, 3)
shape of X_test (10000, 32, 32, 3)
shape of y_train (50000, 1)
shape of y_test (10000, 1)


In [3]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(X_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(X_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(X_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(X_test[ix]/255.0)
        Y2_test.append(y_test[ix])

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [4]:
#One-hot encoding the labels
X1_train = np.asarray(X1_train).reshape(( len(X1_train), 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((len(X1_test),32, 32, 3))
X2_train = np.asarray(X2_train).reshape((len(X2_train),32, 32, 3))
X2_test = np.asarray(X2_test).reshape((len(X2_test), 32, 32, 3))

Y1_train = np_utils.to_categorical(np.asarray(Y1_train), 5)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), 5)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), 10)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), 10)

In [5]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [6]:
#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()
#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))
#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(64, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'))
#Add MaxPooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
#Add dropout layer
model.add(tf.keras.layers.Dropout(0.20))
#Flatten the output
model.add(tf.keras.layers.Flatten())
#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add dropout layer
model.add(tf.keras.layers.Dropout(0.5))

#Output layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

W0922 17:42:14.868152 11468 deprecation.py:506] From C:\Users\Public\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
#Compile
model.compile(optimizer='sgd', 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(x1_train, y1_train,
         nb_epoch=10,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

W0922 17:42:15.094539 11468 training.py:593] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 - 40s - loss: 1.5009 - acc: 0.3377 - val_loss: 1.3737 - val_acc: 0.4612
Epoch 2/10
20000/20000 - 10s - loss: 1.3641 - acc: 0.4318 - val_loss: 1.2945 - val_acc: 0.4840
Epoch 3/10
20000/20000 - 10s - loss: 1.2781 - acc: 0.4748 - val_loss: 1.3160 - val_acc: 0.4628
Epoch 4/10
20000/20000 - 10s - loss: 1.2126 - acc: 0.5072 - val_loss: 1.1945 - val_acc: 0.5124
Epoch 5/10
20000/20000 - 10s - loss: 1.1618 - acc: 0.5317 - val_loss: 1.0942 - val_acc: 0.5652
Epoch 6/10
20000/20000 - 10s - loss: 1.1038 - acc: 0.5560 - val_loss: 1.1125 - val_acc: 0.5530
Epoch 7/10
20000/20000 - 10s - loss: 1.0648 - acc: 0.5731 - val_loss: 1.0161 - val_acc: 0.6108
Epoch 8/10
20000/20000 - 10s - loss: 1.0288 - acc: 0.5853 - val_loss: 0.9642 - val_acc: 0.6146
Epoch 9/10
20000/20000 - 10s - loss: 0.9931 - acc: 0.6011 - val_loss: 0.9442 - val_acc: 0.6266
Epoch 10/10
20000/20000 - 10s - loss: 0.9687 - acc: 0.6098 - val_loss: 0.9236 - val_acc: 0.6406


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [9]:
model.fit(x2_train, y2_train,
         nb_epoch=10,
         shuffle=True,
         batch_size=100,
         validation_data=(x2_val, y2_val), verbose=2)

W0922 17:45:04.199405 11468 training.py:593] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: logits and labels must be broadcastable: logits_size=[100,5] labels_size=[100,10]
	 [[{{node loss/output_1_loss/softmax_cross_entropy_with_logits}}]]
	 [[loss/mul/_53]]
  (1) Invalid argument: logits and labels must be broadcastable: logits_size=[100,5] labels_size=[100,10]
	 [[{{node loss/output_1_loss/softmax_cross_entropy_with_logits}}]]
0 successful operations.
0 derived errors ignored.

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [ ]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [ ]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [ ]:
twenty_train.target

In [ ]:
twenty_train.target_names

In [ ]:
twenty_train.data[0:5]

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score